In [1]:
import pandas as pd
from gensim.models import Word2Vec
import os
import time

# 1. Lemmatize ve stem edilmiş verileri oku
df_lemma = pd.read_csv("reviews_lemmatized.csv")
df_stem = pd.read_csv("reviews_stemmed.csv")

# 2. Tokenize et
lemma_corpus = [str(sentence).split() for sentence in df_lemma["cleanedReview"].dropna()]
stem_corpus = [str(sentence).split() for sentence in df_stem["cleanedReview"].dropna()]

# 3. Parametreler
params = [
    {'model_type': 'cbow', 'window': 2, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 2, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 300},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 300}
]

# 4. Model eğitim ve kaydetme
def train_models(corpus, corpus_type):
    print(f"\n {corpus_type.upper()} veri seti ile model eğitimi başlıyor\n")

    # models klasörünü oluştur
    os.makedirs("models0", exist_ok=True)

    for p in params:
        sg = 1 if p['model_type'] == 'skipgram' else 0
        model_name = f"{corpus_type}_{p['model_type']}_win{p['window']}_dim{p['vector_size']}.model"
        model_path = os.path.join("models0", model_name)

        start = time.time()
        model = Word2Vec(
            sentences=corpus,
            vector_size=p['vector_size'],
            window=p['window'],
            sg=sg,
            min_count=1,
            workers=4,
            epochs=10
        )
        end = time.time()
        model.save(model_path)

        print(f" Model: {model_name}")
        print(f"   ➤ Model Türü     : {p['model_type']}")
        print(f"   ➤ Window         : {p['window']}")
        print(f"   ➤ Vektör Boyutu  : {p['vector_size']}")
        print(f"   ➤ Eğitim Süresi  : {round(end - start, 2)} saniye")
        print(f"   ➤ Vocab Uzunluğu : {len(model.wv)}")

        if "music" in model.wv:
            print(f"   ➤ 'music' kelimesine en yakın 5 kelime:")
            for word, sim in model.wv.most_similar("music", topn=5):
                print(f"     {word} ({sim:.3f})")
        else:
            print("   ➤ 'music' kelimesi modelde bulunamadı.")
        print("-" * 60)

# 5. Tüm modelleri sırayla eğit
train_models(lemma_corpus, "lemmatized")
train_models(stem_corpus, "stemmed")



 LEMMATIZED veri seti ile model eğitimi başlıyor

 Model: lemmatized_cbow_win2_dim100.model
   ➤ Model Türü     : cbow
   ➤ Window         : 2
   ➤ Vektör Boyutu  : 100
   ➤ Eğitim Süresi  : 8.84 saniye
   ➤ Vocab Uzunluğu : 28184
   ➤ 'music' kelimesine en yakın 5 kelime:
     songs (0.707)
     musics (0.676)
     song (0.654)
     tracks (0.559)
     stuff (0.521)
------------------------------------------------------------
 Model: lemmatized_skipgram_win2_dim100.model
   ➤ Model Türü     : skipgram
   ➤ Window         : 2
   ➤ Vektör Boyutu  : 100
   ➤ Eğitim Süresi  : 18.2 saniye
   ➤ Vocab Uzunluğu : 28184
   ➤ 'music' kelimesine en yakın 5 kelime:
     musics (0.796)
     musicpodcasts (0.780)
     musicpodcast (0.767)
     musici (0.749)
     musicand (0.733)
------------------------------------------------------------
 Model: lemmatized_cbow_win4_dim100.model
   ➤ Model Türü     : cbow
   ➤ Window         : 4
   ➤ Vektör Boyutu  : 100
   ➤ Eğitim Süresi  : 10.49 saniye
   ➤ V